In [ ]:
!pip install "unstructured[all-docs]"

In [ ]:
!pip install langchain

In [ ]:
!pip install accelerate sentence-transformers bitsandbytes chromadb docx2txt

In [ ]:
!pip install transformers

In [ ]:
!pip install pypdf

In [ ]:
!pip install tesseract
!pip install pytesseract
!apt-get install poppler-utils
!pip install langchain-community
!pip install sentence-transformers
!pip install chromadb

In [ ]:
!pip install flask_cors
!pip install reportlab

In [ ]:
!pip install pyngrok
!pip install Flask

In [ ]:
!sudo apt install tesseract-ocr

In [ ]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader ,UnstructuredCSVLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders.image import UnstructuredImageLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
import json
import torch
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline , BitsAndBytesConfig
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import transformers

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def Data_ingetion(file_path):
  document = []
  for file in os.listdir(file_path):
    if file.endswith(".pdf"):
      pdf_path=file_path+"/"+file
      loader=UnstructuredPDFLoader(pdf_path)
      document.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.docs'):
      doc_path=file_path+"/"+file
      loader=UnstructuredWordDocumentLoader(doc_path)
      document.extend(loader.load())
    elif file.endswith('.jpg'):
      text_path=file_path+"/"+file
      loader=UnstructuredImageLoader(text_path)
      document.extend(loader.load())
    elif file.endswith('.xlsx'):
      text_path=file_path+"/"+file
      loader=UnstructuredExcelLoader(text_path)
      document.extend(loader.load())
    elif file.endswith('.csv'):
      text_path=file_path+"/"+file
      loader=UnstructuredCSVLoader(text_path)
      document.extend(loader.load())
    elif file.endswith('.jpeg'):
      text_path=file_path+"/"+file
      loader=UnstructuredImageLoader(text_path)
      document.extend(loader.load())
    else:
      continue

  return document


def vector_store(document):
  document_splitter=CharacterTextSplitter(separator='}', chunk_size=7000, chunk_overlap=100)
  document_chunks=document_splitter.split_documents(document)
  vectordb=Chroma.from_documents(document,embedding=embeddings, persist_directory='./datas')
  parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
  child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
  vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings,persist_directory='./datas')
  store = InMemoryStore()

  big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
  )
  big_chunks_retriever.add_documents(document_chunks)

  return big_chunks_retriever



In [ ]:
prompt_template = """
<|start_header_id|>user<|end_header_id|>
You are an assistant for answering questions about RAG.
You are given the extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "I do not know." Don't make up an answer.

Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [ ]:
# if User Question ask Answer in table format than you give detail in table_header and table_detail,\n
# where table_header have table header like colume name and table_detail all the detail of table like rows but in json format .
# Question:Give me RBC count of harsh.

# Answer: 9.3mill

# Question:Give me Top 5 country currency in json format and in form of table headers and table details.
# Answer:{
#   "table_headers": ["Currency Name", "Country", "Symbol", "Code", "Exchange Rate"],
#   "table_details": [
#     ["US Dollar", "United States", "$", "USD", "1.00"],
#     ["Euro", "European Union", "€", "EUR", "0.84"],
#     ["Japanese Yen", "Japan", "¥", "JPY", "110.62"],
#     ["British Pound", "United Kingdom", "£", "GBP", "0.72"],
#     ["Swiss Franc", "Switzerland", "CHF", "1.08"]
#   ]
# }

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
docs = Data_ingetion("/content/drive/MyDrive/policy")
retriever = vector_store(docs)
retriever

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
device_map = "auto"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                          bnb_4bit_quant_type="nf4",
                                          bnb_4bit_use_double_quant=True,
                                          bnb_4bit_compute_dtype=torch.bfloat16,
                                          )

model = AutoModelForCausalLM.from_pretrained(model_name,
                                            quantization_config=quantization_config,
                                            device_map="auto",
                                            low_cpu_mem_usage = True,
                                          #  torch_dtype = torch.bfloat16,
                                          # use_auth_token = False,
                                            #load_in_4bit = True,
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name,)
                                          # use_auth_token=False)
terminators = [
  tokenizer.eos_token_id,
  tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

text_generation_pipeline = pipeline(
  model=model,
  tokenizer=tokenizer,
  task="text-generation",
  temperature=0.2,
  do_sample=True,
  repetition_penalty=1.1,
  return_full_text=False,
  max_new_tokens=200,
  eos_token_id=terminators,
  )

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
query = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type_kwargs={"prompt": prompt},
                                             retriever= retriever,    #vectordb.as_retriever(search_kwargs={'k':2}),
                                             verbose=False)


In [ ]:
while True :
  qu = input()
  if qu == "exit":
    break
  else:
    results = query(qu)
    extract = results['result']
    ans = extract.rsplit('[/INST]',1)[-1].strip()
    print(ans)

In [10]:
from flask import Flask , jsonify ,render_template, request ,send_file
import requests
from pyngrok import ngrok
from flask_cors import CORS
import re

In [2]:
import json
import os
file_path = '/content/drive/MyDrive/policy/chat_history.json'
def append_question_answer(question, answer, file_path):

    data = []
    if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
        with open(file_path, "r") as json_file:
            data = json.load(json_file)
    data.append({
        "question": question,
        "answer": answer
    })
    with open(file_path, "w") as json_file:
        json.dump(data, json_file, indent=2)

In [12]:
import ast
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph ,Spacer
from reportlab.lib.styles import getSampleStyleSheet ,ParagraphStyle

def create_pdf(file_path):
    # Create a PDF document
    pdf_filename = "output.pdf"
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)

    # Define styles for the table
    style_table = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.lightgrey),
                              ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
                              ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                              ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                              ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                              ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                              ('GRID', (0, 0), (-1, -1), 1, colors.black)])

    # Define styles for the text
    #style_text = getSampleStyleSheet()["Normal"]
    # Define styles for the text
    style_text = ParagraphStyle(name='Text', fontSize=10)

    # Define styles for the questions
    style_question = ParagraphStyle(name='Question', fontSize=12, )#textColor=colors.blue)

    # Create a list to hold the content for the PDF
    elements = []
    with open(file_path, "r") as json_file:
      data = json.load(json_file)
    # Iterate over each item in the data
    for item in data:
        question = item['question']
        answer = item['answer']

        # Add question as a heading
        elements.append(Paragraph("Question: "+question, style_question))

        # Check if the answer is in tabular format
        try:
            answer_dict = ast.literal_eval(answer)
            if isinstance(answer_dict, dict) and 'table_headers' in answer_dict and 'table_details' in answer_dict:
                # Answer is in tabular format
                headers = answer_dict['table_headers']
                details = answer_dict['table_details']

                # Create a table
                table_data = [headers] + details
                table = Table(table_data)

                # Apply styles to the table
                table.setStyle(style_table)

                # Add table to the PDF
                elements.append(table)
            else:
                # Answer is not in tabular format, add as text
                elements.append(Paragraph("Answer: "+answer, style_text))
        except (SyntaxError, ValueError):
            # Answer is not in tabular format, add as text
            elements.append(Paragraph("Answer: "+answer, style_text))
        elements.append(Spacer(1, 20))
    # Build PDF document
    doc.build(elements)

# Create PDF
file_path = '/content/drive/MyDrive/policy/chat_history.json'
create_pdf(file_path)


In [11]:
app = Flask(__name__) #template_folder="/content/templates")
ngrok.set_auth_token('2ewXWmHQlnn9apwWj3X0A6r3lFi_TbVgKWhGFqhT46PzckbU')
public_url = ngrok.connect(5000).public_url
CORS(app,resources={r"/ask"})


@app.route("/")
def home():
    return "<h1>GFG is great platform to learn</h1>"


@app.route("/ask", methods=['POST'])
def ask():
    text = request.get_data(as_text=True)
    if "table" in text.lower():
      output_string = re.sub(r'\btable\b', 'json', text)
      extract = query(output_string )#+ " and Only Give table_headers and table details in list [do not use '\n'] and any other text")
      extract = extract['result']
      append_question_answer(text, extract,file_path)
      print(extract)
    else:
      bot_response = query(text)
      extract = str(bot_response['result'])
      append_question_answer(text, extract,file_path)
      print(extract)
    return jsonify({'status':'OK','answer':extract})
    # return jsonify({'status':'OK','answer':bot_response.content})


@app.route('/generate_pdf')
def generate_pdf():
    file_path = create_pdf('/content/drive/MyDrive/policy/chat_history.json')  # Adjust the path accordingly
    return send_file(file_path, as_attachment=True)

# Route to handle file sending
# @app.route('/get-file/<filename>')
# def get_file(filename):
#     # Replace 'uploads' with the directory where your files are stored
#     file_path = f'uploads/{filename}'
#     try:
#         return send_file(file_path, as_attachment=True)
#     except FileNotFoundError:
#         return "File not found", 404


print(f"To access the Gloable link please click {public_url}")
app.run(port=5000)

To access the Gloable link please click https://c7b8-34-168-123-199.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on /generate_pdf [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask_cors/extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_function

In [5]:
import json
with open(file_path, "r") as json_file:
    data = json.load(json_file)
    print(data)

[{'question': '"Give me Address and email of Yagnya Deshpande in table format?"', 'answer': 'Here is the answer to your question:\n\n**Table Headers**\n```\n{\n"Name": "",\n"Email": ""\n}\n```\n\n**Table Details**\n```\n[\n{\n"Name": "Miss. Yagnya Deshpande",\n"Email": "pawarhospitalpvtltd@gmail.com"\n},\n{\n"Name": "Dr. Manish Gaikwad",\n"Email": "pawarhospitalpvtltd@gmail.com"\n},\n{\n"Name": "Suraj Hospital",\n"Email": "spirehospitalpune@gmail.com"\n}\n]\n```\n\nPlease note that there are multiple addresses and emails mentioned in the provided documents, so I\'ve included them all in the response. Let me know if you need further assistance!'}]


In [ ]:
data

In [ ]:
def delete_all_data(file_path):
    # Write an empty list to the file
    with open(file_path, "w") as json_file:
        json.dump([], json_file)

    print("All data deleted successfully!")

delete_all_data(file_path)

All data deleted successfully!


In [ ]:
def remove_question(question):
    for item in data:
        if item['question'] == question:
            data.remove(item)
            return True
    return False

# Example: Remove a question
question_to_remove = 'top 5 county curreny in table format'
if remove_question(question_to_remove):
    print(f"Question '{question_to_remove}' removed successfully.")
else:
    print(f"Question '{question_to_remove}' not found in the data.")

# Output the updated data
print(json.dumps(data, indent=2))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map="auto",
                                             low_cpu_mem_usage = True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=200,
    eos_token_id=terminators,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
prompt_template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are Document Expert AI who Know how find Answer from document which present in Vecter store and give Exect Answer of question.
Given the following conversation and a follow up question,
rephrase the follow up question to be a standalone question, in its original language,
if the answer is not in provided context just say, " Enter a valid Question ", don't provide the wrong answer.
that can be used to query a chromadb. This query will be used to retrieve documents with additional context.
Do not provided extra information from the Document
Let me share a couple examples.
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
what is gmail id of Harsh ?
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
hsinghharsh123@gmail.com
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
what is policy no. or policy number of Harsh singh?
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
92000037890400000107
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
what is Hospital name where health claim?
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
Suraj Hospital
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
what is phone number or mobile number of Harsh ?
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
626575425
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
what is skills of Harsh singh?
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
NumPy, Object Oriented Programming, SQL, HTML, PyCharm, Jupyter Notebook, VS code, Git and GitHub, JIRA
<|eot_id|>

Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>


"""

prompt = PromptTemplate(template = prompt_template,input_variables=["context", "question"],)


In [ ]:
# question = "What is RBC count of miss yagnya deshpande?"
# rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Based on the provided report, I can see that the patient's name is Miss Yagnya Deshpande, and the report is from Pawar Multispeciality Hospital & Diagnostic Centre Pvt. Ltd.\n\nAccording to the report, the RBC count is mentioned as 2.8 mil./Cmm, which falls within the reference range of 4.5 -6.5 mil./Cmm.\n\nSo, the answer to your question is: The RBC count of Miss Yagnya Deshpande is 2.8 mil./Cmm."

In [ ]:
query = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type_kwargs={"prompt": prompt},
                                    chain_type="stuff",
                                             retriever= retriever,    #vectordb.as_retriever(search_kwargs={'k':2}),
                                             verbose=False)

In [ ]:
query("Give me Total Amount of miss Yagnya deshpande ?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'Give me Total Amount of miss Yagnya deshpande ?',
 'result': "The total amount of Miss Yagnya Deshpande's bill is ₹ 31755.00."}

In [ ]:
gh =query("Give me detail of this name is yagnya and policy no. 92000034230400000103 ?")
gh['result']

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Based on the provided context and extracted parts of the document, I can try to answer your question.\n\nThe question is: "Give me detail of this name is yagnya and policy no. 92000034230400000103?"\n\nFrom the given information, we can see that the name is Yagnya, and she is a 22-year-old female who has taken out a health insurance policy with policy number 92000034230400000103. The policy was started on June 1, 2023, and the claimant is seeking cashless hospitalization for her illness, which is diagnosed as acute diarrhea and gastroenteritis of presumed infectious origin.\n\nThe relevant clinical findings mentioned in the report include:\n\n* Haemoglobin level: 9.3 gm%\n* Total WBC Count: 5100 /Cmm\n* RBC count: 2.8 mil./Cmm\n* Differential Neutrophils: 64%\n* Lymphocytes: '

In [ ]:
while True :
  qu = input()
  if qu == "exit":
    break
  else:
    results = query(qu)
    # extract = results['result']
    # ans = extract.rsplit('[/INST]',1)[-1].strip()
    print(results)

what is RBC count of miss yagnya deshpande?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'what is RBC count of miss yagnya deshpande?', 'result': 'Hi there!\n\nAccording to the report, the RBC (Red Blood Cell) count of Miss Yagnya Deshpande is 2.8 million per cubic millimeter (mil./Cmm), which falls within the reference range of 4.5-6.5 mil./Cmm.\n\nLet me know if you have any further questions!'}
What is the policy number of yagnya ?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'What is the policy number of yagnya ?', 'result': "I've searched through the provided context and extracted parts of the document. According to the information, the policy number of Yagnya is **92000034230400000103**."}
does any parasites are seen in blood reports of yagnya deshpande?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'does any parasites are seen in blood reports of yagnya deshpande?', 'result': 'Based on the report, I can tell you that according to the Peripheral blood Smear (PBS), Parasites were NOT seen. The report specifically states "Parasites : Not seen" under the PBS section.'}
Is viral gastro enteritis and hepatitis correct for yagnya?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'query': 'Is viral gastro enteritis and hepatitis correct for yagnya?', 'result': "Hello! I'm happy to help you with your query.\n\nAccording to the discharge summary, Miss. Yagnya Deshpande was diagnosed with Viral Gastro-enteritis with Hepatitis during her hospitalization at Suraj Hospital. \n\nSo, to answer your question, yes, Viral Gastro-enteritis and Hepatitis were indeed the correct diagnoses for Yagnya."}


KeyboardInterrupt: Interrupted by user